In [ ]:
# Cell 0
# If at all possible please test locally or on the private tbears server. The Testnet
# is becoming cluttered with many deployments of Balanced contracts.
# Note that running on the private tbears server will require the number of top P-Reps 
# be set to 4 in the staking contract or it will fail to deploy.

network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"gangnam": {"iconservice": "https://gicon.net.solidwallet.io", "nid": 3},
"hannam": {"iconservice": "https://hannam.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"sejong":  {"iconservice": "https://sejong.net.solidwallet.io", "nid":83},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [ ]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
TEST_ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"
MAIN_ORACLE = "cxe647e0af68a4661566f5e9861ad4ac854de808a2"
SEJONG_ORACLE = "cx9a94ee6152983d1d4a5777cfd989d403855535b7"
BALANCED_TEST = "hx3f01840a599da07b0f620eeae7aa9c574169a4be"
CUSTOM_ORACLE = "cx236877888d2dd48ab327612146254796e516e71a"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [ ]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [ ]:
deployer_wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")

print(icon_service.get_balance(deployer_wallet.get_address())/10**18)

print(deployer_wallet.get_address())

In [ ]:
with open("keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("keystores/balanced_test.json", key_data)

print(btest_wallet.get_address())
print(btest_wallet.get_private_key())

In [ ]:
with open("keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
staking_wallet = KeyWallet.load("keystores/balanced_test.json", key_data)

print(icon_service.get_balance(staking_wallet.get_address())/10**18)

print(staking_wallet.get_address())

In [ ]:
# Cell 5a
## Testnet Sejong
contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx3d9cdc89448f095d8235499a05bdcadc7942551f'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx6d00da94e42d98c322bbc97c8e44e8e7d6edff23'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxcc821aa585b7150e715afcdc6623b5c8d5ecceed'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx100ad9e1dd6baac29d891344047d40698e963445'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cx5281a361cc9b7e8a6ab9065c5f0066fecc22f66e'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx9ca68736bb08b637a71793d465104844642ba300'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx9104db9b4ecebdc93e7d0b1a965848fb3422c8e0'},
 'rebalancing': {'zip': 'core_contracts/rebalancing.zip',
  'SCORE': 'cx65bde5a57809bfa2c147114182a3e264177f306d'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx727f084337a0d3440d326c6e0d9340b76f874db6'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx9a94ee6152983d1d4a5777cfd989d403855535b7'},
 'router': {'zip': 'core_contracts/router.zip',
  'SCORE': 'cxc6873b180d4bb28a0fa62b89878e11a7ea1706b7'},
 'feehandler': {'zip': 'core_contracts/feehandler.zip',
  'SCORE': 'cxaebd64b955a2011289c682f7e67f860f6a117df7'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx4c3c561d61070ef4afc2ec833cb9aa9f58a5e7a6'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx3ac48a2aabe4888603046646cbf63993ca762653'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx5094107cdd3c82153e07e093a057d05c21a19f15'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx986b46fbe1708b90b0bb37e329e630fc970bc35e'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx8c904928dac21720734afff0deb64f86efecb95d'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx53db08d9b753a5784a5d6a9cf049fc3b228003e3'},
 'stakedLp': {'zip': 'core_contracts/stakedLp.zip',
  'SCORE': 'cx53db08d9b753a5784a5d6a9cf049fc3b228003e3'}}

In [ ]:
# Private Tbears
contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx3d9cdc89448f095d8235499a05bdcadc7942551f'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx6d00da94e42d98c322bbc97c8e44e8e7d6edff23'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxcc821aa585b7150e715afcdc6623b5c8d5ecceed'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx100ad9e1dd6baac29d891344047d40698e963445'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cx5281a361cc9b7e8a6ab9065c5f0066fecc22f66e'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx9ca68736bb08b637a71793d465104844642ba300'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx9104db9b4ecebdc93e7d0b1a965848fb3422c8e0'},
 'rebalancing': {'zip': 'core_contracts/rebalancing.zip',
  'SCORE': 'cx65bde5a57809bfa2c147114182a3e264177f306d'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx727f084337a0d3440d326c6e0d9340b76f874db6'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx236877888d2dd48ab327612146254796e516e71a'},
 'router': {'zip': 'core_contracts/router.zip',
  'SCORE': 'cxc6873b180d4bb28a0fa62b89878e11a7ea1706b7'},
 'feehandler': {'zip': 'core_contracts/feehandler.zip',
  'SCORE': 'cxaebd64b955a2011289c682f7e67f860f6a117df7'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx4c3c561d61070ef4afc2ec833cb9aa9f58a5e7a6'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx3ac48a2aabe4888603046646cbf63993ca762653'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx5094107cdd3c82153e07e093a057d05c21a19f15'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx986b46fbe1708b90b0bb37e329e630fc970bc35e'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx8c904928dac21720734afff0deb64f86efecb95d'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx53db08d9b753a5784a5d6a9cf049fc3b228003e3'},
 'stakedLp': {'zip': 'core_contracts/stakedLp.zip',
  'SCORE': 'cx53db08d9b753a5784a5d6a9cf049fc3b228003e3'}}

In [ ]:
# new balanced address for UI test

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxa8e5f6263b8403b24c935ee255e06be84cb45195'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx5ce1ed484edbf4332001ca40e5760acb64444a46'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxbde2a33bfadfad5703f076ec554453fcf21abc78'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cxb843d3a4935ca27911d3c1a2112095f512ff9d56'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxea58bbaabbad19f2f75a3aebcb97b088e3cbc9e9'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx9f782c71811566bff738263931b01bf6844ab8e3'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx9604fc2b833969260aea27373700e1547b69d34a'},
 'rebalancing': {'zip': 'core_contracts/rebalancing.zip',
  'SCORE': 'cxb5e84abcd8ab014ca0a695fa5b3f23f7ff3ac138'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxdbe2036fd326eb04d9f1f336326eefb6ea558289'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx386d9c545dfb74a0ee3441b05f011b75dbc9b7a9'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxc79530e0e4081e2e898bac7baa0075fcf71e55c1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx8c2cf0096636feb17bae2bb018b26dc87afb7bcf'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx692018cec4df32bed6b03922f1a3e42c7a684e0b'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cxf1ec0f7df577e30328e477eddbffca35f9acacac'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cxeed0ccc4ee5781abf65208178572e39fd6e49f53'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx7eb251b924e074d8a86f1a388db7aa7a3957b117'}}

In [ ]:
# sejong reward test
contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx65c5a618e702d3cdf61d06eefc18a09051223445'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx85308dab5a08e1d89f8bc0411d4b6c15d5b96389'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxebfafbcc62fda444786d20ce6b53b1b698f9ca6e'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx8dca2895370224748fce495b2ccd41bbe698bdf4'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cx146ff22d0d04b8bf2a17d18525bb2fb3fae3b5de'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxbb5a6e4946ed4c763cda8e3ce295196b52564057'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx6d34cd2bfd5cf80647fa301530c9c687ea6eb4c4'},
 'rebalancing': {'zip': 'core_contracts/rebalancing.zip',
  'SCORE': 'cx698be4ae952c9c92a3bcef787a0bdd2a0f27a9e4'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx44e95ad5ab3661ead505d0ac40f3c8f51085e59b'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx0999df3af49a227fc8799f0d78d19cd8fe4bb76d'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxe4b51838552855c00d7f14b14b8f56b626a9e78c'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cxffcd3f9aedf58bcc07d83dd602dae8733407595d'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx4f5e21a99588dedaaac814316e17de9243102884'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx61943f594005f398942463316df655e010e5c1b8'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx8eb4032236bfa4b5f18141d55c5d7fee8ee46e71'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx35cd9d93a8ed19217a63244f0d8b0b201e7668b1'}}


In [ ]:
# mainet -equivalent-contracts-in -sejong for router 
contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx38ce9792c5eec50bdc90886241ac2a1e75c89291'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxf8b1e599c4bcdb1c05e820ba12724dfa2c4bcb12'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx5deab95398193256632e76b96502f2854843ef08'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx09d229b3b09cbf13772562a02d08c3320ebe2723'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cx499e9f810a961a97fe21e05fc1bf6a3641cca610'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxdfafcd3b50c03b234ac77392632746f56367a7b2'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cxd3a1c00261687586a1edbe4fe0d56ab91ab8f21c'},
 'rebalancing': {'zip': 'core_contracts/rebalancing.zip',
  'SCORE': 'cx73c82ad8b04761458d208c38a95b8cb0c29a88f2'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxd11b8c885ffee1c37b2c766ee19d8c9547d29393'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx9a94ee6152983d1d4a5777cfd989d403855535b7'},
 'router': {'zip': 'core_contracts/router.zip',
  'SCORE': 'cx8c0e1d425c390689843dcac5fe26681e20e10cca'},
 'feehandler': {'zip': 'core_contracts/feehandler.zip',
  'SCORE': 'cx5be90971cfcdd1b6f1fee757aa877fa70d4cc21b'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx21ecdbdd5cc8e16adec6058d5db8fdc527018352'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx61100013294772b7ec3890f62dfd6911c48e90db'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cxc2fbb39e32e91825663c75bf17d9cbdcde3b0d24'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx467857800522e47a82c0d164599f6ed28503072e'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cxb353f7d0b4ed2e7321e22545e799dabe4c197711'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx39531098788896d7bee7c775b0bfd833a7ed684b'}}

In [ ]:
# mainet -equivalent-contracts-in -sejong for contineous rewards
contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx1d51fea563aa5ebc0ed2308438d2aefbb05bf951'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx68dda6f58efca9c5ab2e32027ac35449ea55f298'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx7bce2c44f3859373663851d8c1c77c75d5c31235'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cxa3b7b407226d2ef179910db5c1f5519d72bf8008'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxeeecc48237548c533818037af2fce7b7bc89f35c'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx9cbfd71ba68c6c8887530830d24e26f5b40d7bfc'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx30e6e063c82454c7407b40805d91adbe056f5dea'},
 'rebalancing': {'zip': 'core_contracts/rebalancing.zip',
  'SCORE': 'cx3fff67c71aa5282f17461281481c160c2905dbdf'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx133f62f32ab8d2da7c11bb4d156fdecce1627069'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx9a94ee6152983d1d4a5777cfd989d403855535b7'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx8ed8fecc73cf0963f1a97a23acb553c137ce8594'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cxc6cc4bce9c6ff5639cdedda8707f293bf711bf8d'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx9e98f68e325a2ad0fe13554a57fc30e8a431e64b'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx5841c948f62acf7da76cd8627fc7740bd58cbe71'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cxcb6c02403c70852e9f4f0d108afc24b474bf1285'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cxf83ad1bfa1a1053e9c5a0cf5f7c1c44e037e3cce'}}

In [ ]:
c ={'loans': {'zip': 'core_contracts/loans.zip', 'SCORE': 'cx1d51fea563aa5ebc0ed2308438d2aefbb05bf951'}, 'staking': {'zip': 'core_contracts/staking.zip', 'SCORE': 'cx68dda6f58efca9c5ab2e32027ac35449ea55f298'}, 'dividends': {'zip': 'core_contracts/dividends.zip', 'SCORE': 'cx7bce2c44f3859373663851d8c1c77c75d5c31235'}, 'reserve': {'zip': 'core_contracts/reserve.zip', 'SCORE': 'cxa3b7b407226d2ef179910db5c1f5519d72bf8008'}, 'daofund': {'zip': 'core_contracts/daofund.zip', 'SCORE': 'cxeeecc48237548c533818037af2fce7b7bc89f35c'}, 'rewards': {'zip': 'core_contracts/rewards.zip', 'SCORE': 'cx9cbfd71ba68c6c8887530830d24e26f5b40d7bfc'}, 'dex': {'zip': 'core_contracts/dex.zip', 'SCORE': 'cx30e6e063c82454c7407b40805d91adbe056f5dea'}, 'rebalancing': {'zip': 'core_contracts/rebalancing.zip', 'SCORE': 'cx3fff67c71aa5282f17461281481c160c2905dbdf'}, 'governance': {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx133f62f32ab8d2da7c11bb4d156fdecce1627069'}, 'oracle': {'zip': 'core_contracts/oracle.zip', 'SCORE': 'cx0999df3af49a227fc8799f0d78d19cd8fe4bb76d'}, 'sicx': {'zip': 'token_contracts/sicx.zip', 'SCORE': 'cx8ed8fecc73cf0963f1a97a23acb553c137ce8594'}, 'bnUSD': {'zip': 'token_contracts/bnUSD.zip', 'SCORE': 'cxc6cc4bce9c6ff5639cdedda8707f293bf711bf8d'}, 'bnXLM': {'zip': 'token_contracts/bnXLM.zip', 'SCORE': 'cx9e98f68e325a2ad0fe13554a57fc30e8a431e64b'}, 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip', 'SCORE': 'cx5841c948f62acf7da76cd8627fc7740bd58cbe71'}, 'baln': {'zip': 'token_contracts/baln.zip', 'SCORE': 'cxcb6c02403c70852e9f4f0d108afc24b474bf1285'}, 'bwt': {'zip': 'token_contracts/bwt.zip', 'SCORE': 'cxf83ad1bfa1a1053e9c5a0cf5f7c1c44e037e3cce'}}
c

In [ ]:
# Cell 6
# Define deploy and send_tx functions

def send_icx(to, amount, wallet):
    transaction = TransactionBuilder()\
        .from_(wallet.get_address())\
        .to(to)\
        .value(amount)\
        .step_limit(1000000) \
        .nid(NID) \
        .nonce(2) \
        .version(3) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    return icon_service.send_transaction(signed_transaction)


def compress():
    """
    Compress all SCORE folders in the core_contracts and token_contracts folders.
    Make sure the oracle address is correct.
    """
    deploy = list(contracts.keys())[:]
    for directory in {"core_contracts", "token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'
    if network == "yeouido":
        contracts['oracle']['SCORE'] = TEST_ORACLE
    elif network == "mainnet":
        contracts['oracle']['SCORE'] = MAIN_ORACLE
    elif network == "custom":
        contracts['oracle']['SCORE'] = CUSTOM_ORACLE
    elif network == "sejong":
        contracts['oracle']['SCORE'] = SEJONG_ORACLE

                    

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 3000000000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res

def send_tx(dest, value, method, params, wallet, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Calling {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    if _print:
        print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        if _print:
            print(f'Failure: {res["failure"]}')
    print(res)
    return res

def fast_send_tx(dest, value, method, params, wallet, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Calling {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

def deploy_all(wallet, staking_wallet):
    """
    Compress and Deploy all SCOREs.
    """
    compress()
    
    deploy = list(contracts.keys())[:]    
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')
    if network == "mainnet":
        if 'bnXLM' in deploy:
            deploy.remove('bnXLM')
        if 'bnDOGE' in deploy:
            deploy.remove('bnDOGE')

    results = {}
    res = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    results[f'{contracts["governance"]}|deploy|{{}}'] = res
    governance = res.get('scoreAddress', '')
    contracts['governance']['SCORE'] = governance
    params = {'_governance': governance}
    for score in deploy:
        res = deploy_SCORE(contracts[score], params, wallet, 0)
        results[f'{contracts[score]}|deploy|{params}'] = res
        contracts[score]['SCORE'] = res.get('scoreAddress', '')

    res = deploy_SCORE(contracts['staking'], {}, staking_wallet, 0)
    results[f'{contracts["staking"]}|deploy|{{}}'] = res
    contracts['staking']['SCORE'] = res.get('scoreAddress', '')

    params = {'_admin': contracts['staking']['SCORE']}
    res = deploy_SCORE(contracts['sicx'], params, staking_wallet, 0)
    results[f'{contracts["sicx"]}|deploy|{params}'] = res
    contracts['sicx']['SCORE'] = res.get('scoreAddress', '')
    
    return results

    
def config_balanced(wallet, staking_wallet):
    """
    Configure all SCOREs before launch.
    """
    config = list(contracts.keys())[:]
    config.remove('governance')
    config.remove('bnDOGE')
    config.remove('bnXLM')
    config.remove('rebalancing')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}
    txns = [{'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'configureBalanced', 'params': {}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results

def launch_balanced(wallet, staking_wallet):
    """
    Launch Balanced, turn on staking management, and set delegation for sICX on the Loans contract.
    """
    if network == "custom":
        preps = {
            "hx9eec61296a7010c867ce24c20e69588e2832bc52",  # ICX Station
            "hx000e0415037ae871184b2c7154e5924ef2bc075e"}  # iBriz-ICONOsphere
    elif network == "yeouido":
        preps = {
            "hx23823847f593ecb65c9e1ea81a789b02766280e8",  # ICX Station
            "hxe0cde6567eb6529fe31b0dc2f2697af84847f321",  # iBriz-ICONOsphere
            "hx83c0fc2bcac7ecb3928539e0256e29fc371b5078",  # Mousebelt
            "hx48b4636e84d8c491c88c18b65dceb7598c4600cc",  # Parrot 9
            "hxb4e90a285a79687ec148c29faabe6f71afa8a066"}  # ICONDAO
    elif network == "mainnet":
        preps = {
            "hxfba37e91ccc13ec1dab115811f73e429cde44d48",  # ICX Station
            "hx231a795d1c719b9edf35c46b9daa4e0b5a1e83aa",  # iBriz-ICONOsphere
            "hxbc9c73670c79e8f6f8060551a792c2cf29a8c491",  # Mousebelt
            "hx28c08b299995a88756af64374e13db2240bc3142"}  # Parrot 9
    elif network == "sejong":
        preps = {
            "hx0f432ad57d1c851860a30b131c0843fb4f809650",  # ICX Station
            "hxf434a1b1f5270d4a028a4f737fa931dd70d7e2ec"}  # iBriz-ICONOsphere
    else:
        return

    txns = [{'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'toggleStakingOn', 'params': {}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'delegate',
             'params': {'_delegations': [{'_address': prep, '_votes_in_per': str(100 * ICX // len(preps))} for prep in preps]}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results
   

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Reading {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(btest_wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    result = icon_service.call(call)
    if _print:
        print(result)
    return result


In [ ]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

if network == 'sejong':
    confirm = 'Yes'
else:
    confirm = input(f'Deploying Balanced to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    results = {}
    deploy_all(btest_wallet, staking_wallet)
    print('------------------------------------------------------------------------------------------------------------------')
    print(contracts)
    print('----------Contracts for Testing UI--------------------------------------------------------------------------------')
    print(get_scores_json(contracts))

In [ ]:
# Cell 7a
# Configure Balanced

config_results = config_balanced(btest_wallet, staking_wallet)
print(config_results)

In [ ]:
# Cell 7b
# Launch Balanced
# We may want to make this a payable method and have the governance SCORE borrow bnUSD,
# start and name the sICXbnUSD market, and add it as a rewards DataSource.

launch_results = launch_balanced(btest_wallet, staking_wallet)
print(launch_results)

In [ ]:
# Cell 8
# Deploy or Update a single SCORE

contract_name = 'dex'
update = 1
params = {}
if update == 0 and contract_name != 'governance':
    params = {'_governance': contracts['governance']['SCORE']}

compress()
contract = contracts[contract_name]
if network == 'custom':
    confirm = 'Yes'
else:
    confirm = input(f'{"Updating" if update else "Deploying"} {contract_name} with params: {params} to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    res = deploy_SCORE(contract, params, btest_wallet, update)
print(res['scoreAddress'])

In [ ]:
transaction = TransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(20 * ICX)\
    .step_limit(30000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
send_tx('loans', 250 * ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 50 * ICX}, btest_wallet)

In [ ]:
send_tx('rewards', 0, 'claimRewards', {}, btest_wallet)

In [ ]:
call_tx('rewards', "getBalnHolding", {'_holder': btest_wallet.get_address()})

In [ ]:
# dex swap
def swap(from_token, to_token, amount,wallet):
    data2 = json.dumps({"method": "_swap", "params": {"toToken":str(to_token)}})
    params = {'_to': contracts['dex']['SCORE'], '_value': amount, '_data': data2.encode()}
    transaction = CallTransactionBuilder()\
        .from_(btest_wallet.get_address())\
        .to(from_token)\
        .step_limit(10000000)\
        .value(0)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    return get_tx_result(tx_hash)
res = swap(contracts['bnUSD']['SCORE'], contracts['sicx']['SCORE'],500*10**18,btest_wallet)
print(res)

In [ ]:

send_tx('governance', 0, 'setRebalancing', {'_address': contracts['rebalancing']['SCORE']}, btest_wallet)
send_tx('rebalancing', 0, 'setGovernance', {'_address': contracts['governance']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'rebalancingSetSicx', {'_address': contracts['sicx']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'rebalancingSetBnusd', {'_address': contracts['bnUSD']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'rebalancingSetLoans', {'_address': contracts['loans']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'rebalancingSetDex', {'_address': contracts['dex']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'setLoansRebalance', {'_address': contracts['rebalancing']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'setLoansDex', {'_address': contracts['dex']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'setRebalancingThreshold', {'_value': 5 * 10 ** 17}, btest_wallet)
send_tx('governance', 0, 'setMaxSellAmount', {"_sicx_value": 1000 *10 ** 18, "_bnusd_value": 1000 *10 ** 18}, btest_wallet)

In [ ]:
send_tx('rewards', 0, 'claimRewards', {}, btest_wallet)


In [ ]:
# create sicx/bnUSD market
send_tx('governance', 500*10**18, 'createBnusdMarket', {}, btest_wallet)


In [ ]:
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to("cxdc519895ef110220db2442ff1b3223182304b68a")\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_id':1, '_name': 'sICX/ICX'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(btest_wallet.get_address())\
                    .to('cx4342802efce67d2441e69b062b1f0110b6a6f820')\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:

call = CallBuilder().from_(btest_wallet.get_address())\
                    .to(contracts['rebalancing']['SCORE'])\
                    .method('getRebalancingStatus')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# add iusdc/usds as quote coin

send_tx('governance', 0, 'dexAddQuoteCoin', {'_address': 'cxb2c075e9130440dd64e6bfd3fe09e5c629f6e183'}, btest_wallet)
send_tx('governance', 0, 'dexAddQuoteCoin', {'_address': 'cx2e008873de78f5bf82008a46b529d94924d9b6ae'}, btest_wallet)


In [ ]:
call_tx('baln', "balanceOf", {'_owner': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'})

In [ ]:
send_tx('baln', 0, 'transfer', {'_to': 'cx133f62f32ab8d2da7c11bb4d156fdecce1627069', '_value': 200*10**18}, btest_wallet)


In [ ]:
# governance contract needs to have baln to create the market

send_tx('governance', 0, 'createBalnMarket', {'_bnUSD_amount': 20*10**18, '_baln_amount': 10*10**18}, btest_wallet)


In [ ]:
send_tx('governance', 500*10**18, 'createBnusdMarket', {}, btest_wallet)


In [ ]:
call = CallBuilder().from_(btest_wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
curl --location --request POST 'https://sejong.net.solidwallet.io/api/v3d' \
--header 'Content-Type: application/json' \
--data-raw '{
    "jsonrpc": "2.0",
    "method": "debug_getTrace",
    "id": 1234,
    "params": {
        "txHash": "0x7061e0883f4fb9ce7a53ade5fd41f77d8683a5c590c739d057210d76b2823954"
    }
}'

In [ ]:
# transfer plain icx to sicx/icx pool
send_icx('hx177b10efe3018961a405cc7c20ee811c552831a5', 30000*10**18, btest_wallet)

In [ ]:
call = CallBuilder().from_(btest_wallet.get_address())\
                    .to('cx683c78174c740c3c6215de0c3ead1fad024324d6')\
                    .method('balanceOf')\
                    .params({'_owner': btest_wallet.get_address()})\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
call_tx('bnUSD', 'balanceOf', {'_owner': btest_wallet.get_address()})

In [ ]:
# Cell 36 add omm

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 500 * 10**6, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to('cxb2c075e9130440dd64e6bfd3fe09e5c629f6e183')\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Add assets to existing pool
sICX_contribute_amount = int(100 * ICX)
bnUSD_contribute_amount = int(100 * ICX)
# bnUSD_amount = 80000 * ICX
# icx_price = 2

txns = [
#     {'contract': 'staking', 'value': 300 *10**18, 'method': 'stakeICX', 'params': {}},
#         {'contract': 'loans', 'value': 200000 *ICX, 'method': 'depositAndBorrow', 'params': {'_asset': 'bnUSD', '_amount': bnUSD_amount}},
#         {'contract': 'iusdc', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': sICX_contribute_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'bnUSD', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': bnUSD_contribute_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'dex', 'value': 0, 'method': 'add', 'params': {'_baseToken': contracts['baln']['SCORE'], '_quoteToken': contracts['sicx']['SCORE'], '_baseValue': sICX_contribute_amount, '_quoteValue': bnUSD_contribute_amount}}
]

supply_liquidity_results = {}
for tx in txns:
    res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], btest_wallet)
    supply_liquidity_results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

In [ ]:
# get addresses

call = CallBuilder().from_(btest_wallet.get_address())\
                    .to(contracts['governance']['SCORE'])\
                    .method('getAddresses')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
send_tx('governance', 0, 'setContractAddresses', {}, btest_wallet)


In [ ]:
# set address
add = {'baln': 'cx8c904928dac21720734afff0deb64f86efecb95d', 'bnUSD': 'cx3ac48a2aabe4888603046646cbf63993ca762653', 'bwt': 'cx53db08d9b753a5784a5d6a9cf049fc3b228003e3', 'daofund': 'cx5281a361cc9b7e8a6ab9065c5f0066fecc22f66e', 'dex': 'cx9104db9b4ecebdc93e7d0b1a965848fb3422c8e0', 'dividends': 'cxcc821aa585b7150e715afcdc6623b5c8d5ecceed', 'feehandler': 'cxaebd64b955a2011289c682f7e67f860f6a117df7', 'loans': 'cx3d9cdc89448f095d8235499a05bdcadc7942551f', 'oracle': 'cx8f87a4ce573a2e1377545feabac48a960e8092bb', 'rebalancing': 'cx65bde5a57809bfa2c147114182a3e264177f306d', 'reserve': 'cx100ad9e1dd6baac29d891344047d40698e963445', 'rewards': 'cx9ca68736bb08b637a71793d465104844642ba300', 'router': 'cxc6873b180d4bb28a0fa62b89878e11a7ea1706b7', 'sicx': 'cx4c3c561d61070ef4afc2ec833cb9aa9f58a5e7a6', 'staking': 'cx6d00da94e42d98c322bbc97c8e44e8e7d6edff23'}
send_tx('governance', 0, 'setAddresses', {'_addresses': add}, btest_wallet)


In [ ]:
from iconservice import Address
send_tx('feehandler', 0, 'setAcceptedDividendTokens', {'_tokens': ["cx21ecdbdd5cc8e16adec6058d5db8fdc527018352",
            "cx61100013294772b7ec3890f62dfd6911c48e90db",
            "cxb353f7d0b4ed2e7321e22545e799dabe4c197711"]}, btest_wallet)


In [ ]:
send_tx('governance', 0, 'setFeeProcessingInterval', {'_interval': 100}, btest_wallet)


In [ ]:
# set route
fromToken = "cxb2c075e9130440dd64e6bfd3fe09e5c629f6e183"
toToken = "cxb45058d398614a7c8cdf7be6f556fa0b39399799"     
path = [ "cx683c78174c740c3c6215de0c3ead1fad024324d6",
        "cx0e706eca3552a6e607095319f4ad8cea37e779d4",
        "cx041714d034919c8456d3606f8766f0169e35cb8e",
        "cxb45058d398614a7c8cdf7be6f556fa0b39399799"
        ]
send_tx('governance', 0, 'setRoute', {'_fromToken':fromToken, '_toToken':toToken, '_path': path}, btest_wallet)


In [ ]:
send_tx('governance', 0, 'enable_fee_handler', {}, btest_wallet)

In [ ]:
# delete route
fromToken = "cxb2c075e9130440dd64e6bfd3fe09e5c629f6e183"
toToken = "cx041714d034919c8456d3606f8766f0169e35cb8e"     
send_tx('fees', 0, 'deleteRoute', {'_fromToken':fromToken, '_toToken':toToken}, btest_wallet)

In [ ]:
call = CallBuilder().from_(btest_wallet.get_address())\
                    .to(contracts['fees']['SCORE'])\
                    .method('name')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
send_tx('router', 100*ICX, 'route', {'_path':['cx21ecdbdd5cc8e16adec6058d5db8fdc527018352', 'cx61100013294772b7ec3890f62dfd6911c48e90db']}, btest_wallet)

In [ ]:
send_tx('rewards', 0, 'claimRewards', {}, btest_wallet)

In [ ]:
call = CallBuilder().from_(btest_wallet.get_address())\
                    .to(contracts['baln']['SCORE'])\
                    .method('balanceOf')\
                    .params({'_owner': btest_wallet.get_address()})\
                    .build()
result = icon_service.call(call)
print(result)